In [6]:
#import libraries
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np
from sklearn.preprocessing import OrdinalEncoder

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from pathlib import Path
from tqdm import tqdm

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import GRU, Dense, RNN, GRUCell, Input
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import plot_model

import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
  """"Generate data for time series creating windows of given n."""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [10]:
def spacing_window(data,win_space):
  """"creating windows with specific frame."""
  data=data[data.index%win_space == 0]
  return data

In [ ]:
dataset_complete=pd.read_csv('/content/drive/MyDrive/data_gan_final.csv')
ord_enc = OrdinalEncoder()
dataset_complete["Word_code"] = ord_enc.fit_transform(dataset_complete[["Word"]])
dataset_complete=dataset_complete.drop(['Word'], axis=1)

x=max(dataset_complete['ID'])
y=min(dataset_complete['ID'])
print(x)
print(y)
TRAIN=np.array([])
TEST=np.array([])
for i in range (y,x+1):
  dataset=dataset_complete[dataset_complete['ID']== i]
  dataset_temp=dataset[dataset.columns.drop(list(dataset.filter(regex='_r$')))]
  dataset1=dataset.drop(['ID', 'File'], axis=1)
  
  dataset_out=dataset_temp.drop(['ID', 'File'], axis=1)
  
  values1 = dataset1.values
  values_out=dataset_out.values
  n_hours = 30
  n_features=28
  n_out_features = 11
  # # convert frame as time series data
  reframed1 = series_to_supervised(values1, n_hours, 1)
  reframed1=spacing_window(reframed1,10)
  values1 = reframed1.values
  reframed_out = series_to_supervised(values_out, n_hours, 1)
  reframed_out=spacing_window(reframed_out,10)
  values_out = reframed_out.values
  # test and train split
  train = values1
  test=values_out
  n_obs = n_hours * n_features
  
  train = train.reshape((train.shape[0], n_hours, n_features))
  test = test.reshape((test.shape[0], n_hours, n_out_features))
  print(train.shape, test.shape)
  if len(TRAIN)==0:
    TRAIN=train
    TEST=test
    
  else:
    TRAIN=np.concatenate((TRAIN, train))
    TEST=np.concatenate((TEST, test))
    
  print('@@@@@@@@@@@@@@@@@@@@')
  print(TRAIN.shape, TEST.shape)


In [ ]:
data=TRAIN

In [ ]:
seq_len = 30
n_seq = 17
batch_size = 128

In [ ]:
n_windows=len(data)

In [ ]:
data.shape

In [ ]:
real_series = (tf.data.Dataset
               .from_tensor_slices(data)
              #  .shuffle(buffer_size=n_windows)
               .batch(128))
real_series_iter = iter(real_series.repeat())

In [ ]:
def make_random_data():
    while True:
        yield np.random.uniform(low=0, high=1, size=(seq_len, n_seq))

In [ ]:
data1=TEST
data1.shape

In [ ]:
real_series1 = (tf.data.Dataset
               .from_tensor_slices(data1)
              #  .shuffle(buffer_size=n_windows)
               .batch(128))
real_series1_iter = iter(real_series1.repeat())

In [ ]:
random_series = iter(tf.data.Dataset
                     .from_generator(make_random_data, output_types=tf.float32)
                     .batch(batch_size)
                     .repeat())

In [ ]:
# from pickle import dump
# dump(ord_enc, open('ordinal.pkl', 'wb'))

In [ ]:
# from pickle import load
# ord_enc = load(open('ordinal.pkl', 'rb'))
# xx=ord_enc.transform([['my']])
# print(xx)

In [ ]:
hidden_dim = 56
num_layers = 3

In [ ]:
X = Input(shape=[seq_len, 28], name='RealData')
Z = Input(shape=[seq_len, 28], name='RandomData')

In [ ]:
def make_rnn(n_layers, hidden_units, output_units, name):
    """ GRU layers """
    return Sequential([GRU(units=hidden_units,
                           return_sequences=True,
                           name=f'GRU_{i + 1}') for i in range(n_layers)] +
                      [Dense(units=output_units,
                             activation='sigmoid',
                             name='OUT')], name=name)

In [ ]:
embedder = make_rnn(n_layers=3, 
                    hidden_units=hidden_dim, 
                    output_units=hidden_dim, 
                    name='Embedder')
recovery = make_rnn(n_layers=3, 
                    hidden_units=hidden_dim, 
                    output_units=n_seq, 
                    name='Recovery')

In [ ]:
generator = make_rnn(n_layers=3, 
                     hidden_units=hidden_dim, 
                     output_units=hidden_dim, 
                     name='Generator')
discriminator = make_rnn(n_layers=3, 
                         hidden_units=hidden_dim, 
                         output_units=1, 
                         name='Discriminator')
supervisor = make_rnn(n_layers=2, 
                      hidden_units=hidden_dim, 
                      output_units=hidden_dim, 
                      name='Supervisor')

In [ ]:
train_steps = 10000
gamma = 1

In [ ]:
mse = MeanSquaredError()
bce = BinaryCrossentropy()

In [ ]:
H = embedder(X)
X_tilde = recovery(H)

autoencoder = Model(inputs=X,
                    outputs=X_tilde,
                    name='Autoencoder')

In [ ]:
autoencoder.summary()

In [32]:
autoencoder_optimizer = Adam()

In [33]:
@tf.function
def train_autoencoder_init(x):
    with tf.GradientTape() as tape:
        x_tilde = autoencoder(x)
        x.shape
        embedding_loss_t0 = mse(x[:,:,:17], x_tilde)
        e_loss_0 = 10 * tf.sqrt(embedding_loss_t0)

    var_list = embedder.trainable_variables + recovery.trainable_variables
    gradients = tape.gradient(e_loss_0, var_list)
    autoencoder_optimizer.apply_gradients(zip(gradients, var_list))
    return tf.sqrt(embedding_loss_t0)

In [ ]:
for step in tqdm(range(train_steps)):
    X_ = next(real_series_iter)
    next(real_series1_iter)
    step_e_loss_t0 = train_autoencoder_init(X_)
    

In [35]:
supervisor_optimizer = Adam()

In [36]:
@tf.function
def train_supervisor(x):
    with tf.GradientTape() as tape:
        h = embedder(x)
        h_hat_supervised = supervisor(h)
        g_loss_s = mse(h[:, 1:, :], h_hat_supervised[:, :-1, :])

    var_list = supervisor.trainable_variables
    gradients = tape.gradient(g_loss_s, var_list)
    supervisor_optimizer.apply_gradients(zip(gradients, var_list))
    return g_loss_s

In [ ]:
for step in tqdm(range(train_steps)):
    X_ = next(real_series_iter)
    next(real_series1_iter)
    step_g_loss_s = train_supervisor(X_)
    

In [ ]:
E_hat = generator(Z)
H_hat = supervisor(E_hat)
Y_fake = discriminator(H_hat)

adversarial_supervised = Model(inputs=Z,
                               outputs=Y_fake,
                               name='AdversarialNetSupervised')

In [ ]:
adversarial_supervised.summary()

In [ ]:
Y_fake_e = discriminator(E_hat)

adversarial_emb = Model(inputs=Z,
                    outputs=Y_fake_e,
                    name='AdversarialNet')

In [ ]:
adversarial_emb.summary()

In [46]:
X_hat = recovery(H_hat)
synthetic_data = Model(inputs=Z,
                       outputs=X_hat,
                       name='SyntheticData')

In [47]:
X_hat = recovery(H_hat)
synthetic_data = Model(inputs=Z,
                       outputs=X_hat,
                       name='SyntheticData')

In [ ]:
synthetic_data.summary()

In [49]:
def get_generator_moment_loss(y_true, y_pred):
    y_true_mean, y_true_var = tf.nn.moments(x=y_true, axes=[0])
    y_pred_mean, y_pred_var = tf.nn.moments(x=y_pred, axes=[0])
    g_loss_mean = tf.reduce_mean(tf.abs(y_true_mean - y_pred_mean))
    g_loss_var = tf.reduce_mean(tf.abs(tf.sqrt(y_true_var + 1e-6) - tf.sqrt(y_pred_var + 1e-6)))
    return g_loss_mean + g_loss_var

In [50]:
Y_real = discriminator(H)
discriminator_model = Model(inputs=X,
                            outputs=Y_real,
                            name='DiscriminatorReal')

In [ ]:
discriminator_model.summary()

In [52]:
generator_optimizer = Adam()
discriminator_optimizer = Adam()
embedding_optimizer = Adam()

In [53]:
@tf.function
def train_generator(x, z):
    with tf.GradientTape() as tape:
        y_fake = adversarial_supervised(z)
        generator_loss_unsupervised = bce(y_true=tf.ones_like(y_fake),
                                          y_pred=y_fake)

        y_fake_e = adversarial_emb(z)
        generator_loss_unsupervised_e = bce(y_true=tf.ones_like(y_fake_e),
                                            y_pred=y_fake_e)
        h = embedder(x)
        h_hat_supervised = supervisor(h)
        generator_loss_supervised = mse(h[:, 1:, :], h_hat_supervised[:, 1:, :])

        x_hat = synthetic_data(z)
        x=tf.cast(x,tf.float32)

        generator_moment_loss = get_generator_moment_loss(x[:,:,:17], x_hat)

        generator_loss = (generator_loss_unsupervised +
                          generator_loss_unsupervised_e +
                          100 * tf.sqrt(generator_loss_supervised) +
                          100 * generator_moment_loss)

    var_list = generator.trainable_variables + supervisor.trainable_variables
    gradients = tape.gradient(generator_loss, var_list)
    generator_optimizer.apply_gradients(zip(gradients, var_list))
    return generator_loss_unsupervised, generator_loss_supervised, generator_moment_loss

In [54]:
@tf.function
def train_embedder(x):
    with tf.GradientTape() as tape:
        h = embedder(x)
        h_hat_supervised = supervisor(h)
        generator_loss_supervised = mse(h[:, 1:, :], h_hat_supervised[:, 1:, :])

        x_tilde = autoencoder(x)
        embedding_loss_t0 = mse(x[:,:,:17], x_tilde)
        e_loss = 10 * tf.sqrt(embedding_loss_t0) + 0.1 * generator_loss_supervised

    var_list = embedder.trainable_variables + recovery.trainable_variables
    gradients = tape.gradient(e_loss, var_list)
    embedding_optimizer.apply_gradients(zip(gradients, var_list))
    return tf.sqrt(embedding_loss_t0)

In [55]:
@tf.function
def get_discriminator_loss(x, z):
    y_real = discriminator_model(x)
    discriminator_loss_real = bce(y_true=tf.ones_like(y_real),
                                  y_pred=y_real)

    y_fake = adversarial_supervised(z)
    discriminator_loss_fake = bce(y_true=tf.zeros_like(y_fake),
                                  y_pred=y_fake)

    y_fake_e = adversarial_emb(z)
    discriminator_loss_fake_e = bce(y_true=tf.zeros_like(y_fake_e),
                                    y_pred=y_fake_e)
    return (discriminator_loss_real +
            discriminator_loss_fake +
            gamma * discriminator_loss_fake_e)

In [56]:
@tf.function
def train_discriminator(x, z):
    with tf.GradientTape() as tape:
        discriminator_loss = get_discriminator_loss(x, z)

    var_list = discriminator.trainable_variables
    gradients = tape.gradient(discriminator_loss, var_list)
    discriminator_optimizer.apply_gradients(zip(gradients, var_list))
    return discriminator_loss

In [ ]:
step_g_loss_u = step_g_loss_s = step_g_loss_v = step_e_loss_t0 = step_d_loss = 0
count=0
for step in range(train_steps):
    # Train generator (twice as often as discriminator)
    for kk in range(2):
        X_ = next(real_series_iter)
        # Z_ = next(random_series)
        x=next(real_series1_iter)
        if x.shape[0] != 128:
          print(x.shape)
          X_ = next(real_series_iter)
          # Z_ = next(random_series)
          x=next(real_series1_iter)
          count=count+1
        if count==1:
          print(x.shape)
          count=0
        y=next(random_series)
        x = tf.cast(x, tf.float32)
        Z_=tf.concat([x, y], 2)
        # Train generator
        step_g_loss_u, step_g_loss_s, step_g_loss_v = train_generator(X_, Z_)
        # Train embedder
        step_e_loss_t0 = train_embedder(X_)

    X_ = next(real_series_iter)
    x=next(real_series1_iter)
    if x.shape[0] != 128:
      print(x.shape)
      X_ = next(real_series_iter)
      # Z_ = next(random_series)
      x=next(real_series1_iter)
      count=count+1
    if count==1:
      print(x.shape)
      count=0
    y=next(random_series)
    x = tf.cast(x, tf.float32)
    Z_=tf.concat([x, y], 2)
    step_d_loss = get_discriminator_loss(X_, Z_)
    if step_d_loss > 0.15:
        step_d_loss = train_discriminator(X_, Z_)

    if step % 1000 == 0:
        print(f'{step:6,.0f} | d_loss: {step_d_loss:6.4f} | g_loss_u: {step_g_loss_u:6.4f} | '
              f'g_loss_s: {step_g_loss_s:6.4f} | g_loss_v: {step_g_loss_v:6.4f} | e_loss_t0: {step_e_loss_t0:6.4f}')

    

In [ ]:
synthetic_data.save('synthetic_data')

In [ ]:
generated_data = []
for i in range(int(n_windows / batch_size)):
    x=next(real_series1_iter)
    y=next(random_series)
    x = tf.cast(x, tf.float32)
    Z_=tf.concat([x, y], 2)
    d = synthetic_data(Z_)
    generated_data.append(d)

In [ ]:
len(generated_data)

In [ ]:
generated_data = np.array(np.vstack(generated_data))
generated_data.shape